在沙盒环境，对信号衍生处理的三种方式

In [1]:
import os,pdb,itertools,copy,datetime,sys,collections
os.environ['ULTRON_DATA'] = 'keim'

In [2]:
import numpy as np
import pandas as pd

In [3]:
from ultron.env import *

/var/log/ultron/2022-09-28.log


In [4]:
enable_example_env()

2022-09-28 10:43:32,809 - [env.py:67] - ultron - INFO - enable example env will only read /home/kerry/ultron/rom/sandbox/keim


In [5]:
from ultron.strategy.builder.signal import build_ts_signal,calc_weighted_signal,build_cs_signal,build_qs_signal
from ultron.strategy.builder.factor import ts_zscore_normalization,cross_section_scale

#### 读取因子

In [6]:
total_data = pd.read_csv(os.path.join(g_project_data, 'factor.csv'), index_col=0)
total_data['trade_date'] = pd.to_datetime(total_data['trade_date'])
factors_data = total_data[['trade_date','code','WeightShortVolRelTotIntChg','BM_RecentSecond_20D']]
factors_data.head()

,trade_date,code,WeightShortVolRelTotIntChg,BM_RecentSecond_20D
0,2017-10-27,A,-0.037579,-0.023047
1,2017-10-27,AL,-0.012891,0.001133
2,2017-10-27,BU,0.275875,0.069999
3,2017-10-27,C,0.245555,-0.001541
4,2017-10-27,CF,-0.131799,0.004838


#### 时序标准化

In [7]:
def normalization(factors_data, name, bound=3, winsize=120):
    factor_data = factors_data.set_index(['trade_date','code'])[name].dropna().unstack()
    rval = ts_zscore_normalization(data=factor_data, bound=bound, winsize=winsize)
    return rval

In [8]:
factor_data1 = normalization(factors_data=factors_data, name='WeightShortVolRelTotIntChg')
factor_data2 = normalization(factors_data=factors_data, name='BM_RecentSecond_20D')

#### 横截面标准化

In [9]:
factor_data1 = cross_section_scale(factor_data1)
factor_data2 = cross_section_scale(factor_data2)

### 多因子信号合成

#### 构建数据集

In [10]:
signals = collections.OrderedDict()

In [11]:
signals['WeightShortVolRelTotIntChg'] = factor_data1
signals['BM_RecentSecond_20D'] = factor_data2
factor_list = ['WeightShortVolRelTotIntChg','BM_RecentSecond_20D']
factor_weight = [0.5,0.5]

#### 因子加权

In [12]:
weigthed = calc_weighted_signal(signals, factor_list, factor_weight)
weigthed.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,0.019153,0.041251,0.002103,-0.025657,0.032708,0.011506,0.012441,0.020483,-0.001438,-0.003308,...,0.018207,-0.020650,0.012860,-0.012399,0.038972,-0.038019,-0.026032,-0.007904,-0.008203,0.001723
2022-06-17,0.020092,0.031195,0.013501,-0.047116,0.019029,-0.008148,0.021857,0.004359,-0.034689,0.015959,...,0.041486,-0.022772,0.018861,0.002266,0.024402,-0.020301,-0.000903,-0.015956,-0.006656,0.019837
2022-06-20,0.028062,0.035740,0.014332,-0.026098,0.030309,-0.001631,0.035979,0.020550,-0.031629,0.023821,...,0.047478,-0.054444,0.025348,-0.007312,0.022329,-0.013262,-0.002426,-0.012271,0.003843,0.066617
2022-06-21,0.028113,0.037144,0.024982,-0.013389,0.035868,0.002471,0.034907,-0.004691,-0.010700,0.006578,...,0.040327,-0.015019,0.030972,-0.001642,0.082832,-0.030429,-0.014258,-0.015524,-0.004235,0.064214
2022-06-22,0.029034,0.025558,0.023268,0.043909,0.036202,-0.006711,0.033628,-0.013291,-0.032340,0.000139,...,0.047920,-0.018767,0.018534,0.013492,0.084965,-0.031115,-0.005580,-0.013369,0.001593,0.067409


#### 时间序列信号处理

In [13]:
rval = build_ts_signal(data=weigthed, bound=3, winsize=100,
                zscore_normalization=True,
                nominal_invest_weight=True,
                ts_row_scale=True)
rval.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,0.014093,0.043944,0.002180,-0.028822,0.044428,0.012278,0.014320,0.026858,-0.001593,-0.004932,...,0.035934,-0.029168,0.023795,-0.024349,0.079768,-0.034759,-0.046925,-0.008141,-0.016276,0.002284
2022-06-17,0.013145,0.029272,0.012434,-0.046850,0.022739,-0.007743,0.022461,0.005096,-0.034172,0.021384,...,0.072367,-0.028491,0.032241,0.003966,0.043126,-0.016392,-0.001456,-0.014603,-0.011793,0.023368
2022-06-20,0.016737,0.030643,0.012011,-0.023330,0.032978,-0.001418,0.033582,0.021883,-0.028233,0.030323,...,0.073292,-0.061762,0.041745,-0.011738,0.035534,-0.009738,-0.003581,-0.010222,0.006257,0.071516
2022-06-21,0.017086,0.032297,0.021356,-0.012296,0.039581,0.002213,0.033139,-0.005083,-0.009719,0.008701,...,0.061414,-0.016923,0.051544,-0.002695,0.078543,-0.022800,-0.021489,-0.013194,-0.007148,0.068366
2022-06-22,0.017387,0.021752,0.019564,0.039974,0.039047,-0.005966,0.031254,-0.014245,-0.028947,0.000183,...,0.070995,-0.020815,0.029849,0.021902,0.077446,-0.022901,-0.008265,-0.011215,0.002701,0.068627


#### 横截面信号处理

In [14]:
rval = build_cs_signal(data=weigthed,
                    bound=3,
                    winsize=100,
                    zscore_normalization=True,
                    equal_weighted=True)
rval.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,0.029412,0.029412,0.029412,-0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,0.029412,...,0.029412,-0.029412,0.029412,-0.029412,0.029412,-0.029412,-0.029412,0.000000,-0.029412,0.029412
2022-06-17,0.029412,0.029412,0.029412,-0.029412,0.029412,-0.029412,0.029412,0.029412,-0.029412,0.029412,...,0.029412,-0.029412,0.029412,0.029412,0.029412,-0.029412,-0.029412,-0.029412,-0.029412,0.029412
2022-06-20,0.029412,0.029412,0.029412,-0.029412,0.029412,-0.029412,0.029412,0.029412,-0.029412,0.029412,...,0.029412,-0.029412,0.029412,-0.029412,0.029412,-0.029412,-0.029412,-0.029412,0.000000,0.029412
2022-06-21,0.029412,0.029412,0.029412,-0.029412,0.029412,0.000000,0.029412,-0.029412,-0.029412,0.029412,...,0.029412,-0.029412,0.029412,-0.029412,0.029412,-0.029412,-0.029412,-0.029412,-0.029412,0.029412
2022-06-22,0.029412,0.029412,0.029412,0.029412,0.029412,-0.029412,0.029412,-0.029412,-0.029412,-0.029412,...,0.029412,-0.029412,0.029412,0.029412,0.029412,-0.029412,-0.029412,-0.029412,-0.029412,0.029412


#### 分组信号处理

In [16]:
rval = build_qs_signal(data=weigthed,split_strategy_type=[5,4.5,2.5],bound=3, winsize=3)
rval.tail()

code,A,AL,BU,C,CF,CS,CU,EG,FU,HC,...,RU,SA,SF,SM,SR,TA,V,Y,ZC,ZN
trade_date,,,,,,,,,,,,,,,,,,,,,
2022-06-16,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,...,0.000000,-0.071429,0.071429,-0.071429,0.000000,0.000000,-0.071429,0.000000,0.0,0.071429
2022-06-17,0.071429,0.071429,0.000000,0.000000,0.071429,-0.071429,0.000000,0.000000,0.000000,0.000000,...,0.000000,-0.071429,0.071429,0.000000,0.071429,-0.071429,0.000000,-0.071429,0.0,0.071429
2022-06-20,0.000000,0.000000,0.071429,-0.071429,0.071429,-0.071429,0.000000,0.071429,0.000000,0.000000,...,0.071429,0.000000,0.071429,0.000000,0.000000,-0.071429,0.000000,-0.071429,0.0,0.000000
2022-06-21,0.071429,0.000000,0.000000,-0.071429,0.071429,0.000000,0.071429,-0.071429,-0.071429,0.000000,...,0.071429,-0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2022-06-22,0.071429,0.000000,0.000000,0.000000,0.071429,0.000000,0.071429,0.000000,0.000000,-0.071429,...,0.071429,-0.071429,0.000000,0.000000,0.000000,0.000000,-0.071429,0.000000,0.0,0.000000
